In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geopandas as gpd
import os
import pandas as pd

In [ ]:
def clip_drainage_lines(basin_boundary_path, input_file_path, drainage_lines_folder, output_shapefile_path):
    try:
        # Load input basin boundary shapefile
        basin_boundary = gpd.read_file(basin_boundary_path)
        basin_boundary = basin_boundary.to_crs('EPSG:4326')

        # Load the input file
        input_file = gpd.read_file(input_file_path)
        input_file = input_file.to_crs('EPSG:4326')

        # Create an empty GeoDataFrame to store the result
        result_gdf = gpd.GeoDataFrame(geometry=[], crs='EPSG:4326')

        # Iterate over each basin in the input shapefile
        for index, basin in basin_boundary.iterrows():
            # if not basin.geometry.is_valid:
            #     basin.geometry = basin.geometry.buffer(0)  # Attempt to fix invalid geometries

            clipped_basin = gpd.clip(input_file, basin.geometry)

            # Check if the input file intersects with this basin
            if not clipped_basin.empty and not (clipped_basin.geometry.type == 'Point').any():
                # Load drainage lines shapefile for this basin
                drainage_lines_path = os.path.join(drainage_lines_folder, f"{basin['ba_name']}_dl_so.shp")
                drainage_lines = gpd.read_file(drainage_lines_path)
                drainage_lines = drainage_lines.to_crs('EPSG:4326')

                # Clip drainage lines based on the input shapefile boundary
                clipped_lines = gpd.clip(drainage_lines, input_file.geometry)
                if not clipped_lines.empty and not (clipped_lines.geometry.type == 'Point').any():
                    result_gdf = gpd.GeoDataFrame(pd.concat([result_gdf, clipped_lines], ignore_index=True), crs='EPSG:4326')

        if not result_gdf.empty:
            int_columns = ['fid', 'cat', 'type_code', 'network', 'ORDER']
            result_gdf[int_columns] = result_gdf[int_columns].astype(int)
            # Save the final result to a new shapefile
            result_gdf.to_file(output_shapefile_path)
            print("Clipping process completed for:", output_shapefile_path)
        else:
            print("No valid geometries were processed for:", output_shapefile_path)

    except Exception as e:
        print(f"Error processing file {input_file_path}: {e}")

In [ ]:
import numpy as np

In [ ]:
def calculate_drainage_density(mws_path, drainage_line_path, output_path):

    # Define influence factors for stream orders 1 to 11
    influence_factors = [60/385, 55/385, 50/385, 45/385, 40/385, 35/385, 30/385, 25/385, 20/385, 15/385, 10/385]
    # Load watershed shapefile
    watersheds = gpd.read_file(mws_path)

    # Load drainage lines shapefile
    drainage_lines = gpd.read_file(drainage_line_path)

    # changing CRS for length calculation
    drainage_lines = drainage_lines.to_crs(crs=7755)
    watersheds = watersheds.to_crs(crs=7755)
    watersheds['DD'] = None
    watersheds['DD_stream'] = None
    watersheds['str_len_km'] = None

    # Iterate over each watershed
    for index, watershed in watersheds.iterrows():
        # Filter drainage lines within the current watershed
        clipped_drainage_lines = gpd.clip(drainage_lines, watershed.geometry)

        stream_length = {}
        stream_drainage_density = {}

        # Calculate the total area of the current watershed
        area = watershed['area_in_ha'] / 100

        # Iterate over stream orders and calculate drainage density
        for stream_order, influence_factor in zip(range(1, 12), influence_factors):
            # Filter drainage lines for the current stream order
            stream_order_lines = clipped_drainage_lines[clipped_drainage_lines['ORDER'] == stream_order]

            # Calculate the sum of lengths for the current stream order
            total_length_stream_order = stream_order_lines.geometry.length.sum() / 1000

            # Calculate drainage density for the current stream order
            drainage_density = total_length_stream_order * influence_factor * 100 / area

            stream_length[stream_order] = total_length_stream_order
            stream_drainage_density[stream_order] = drainage_density

        # Create new columns in the 'watersheds' GeoDataFrame
        watersheds.at[index, 'DD'] = sum(stream_drainage_density.values())
        watersheds.at[index, 'DD_stream'] = stream_drainage_density
        watersheds.at[index, 'str_len_km'] = stream_length

    # Restoring the original CRS
    drainage_lines = drainage_lines.to_crs(crs=4326)
    watersheds = watersheds.to_crs(crs=4326)


    watersheds['DD'] = watersheds['DD'].astype(float)
    watersheds['DD_stream'] = watersheds['DD_stream'].astype(object)
    watersheds['str_len_km'] = watersheds['str_len_km'].astype(object)

    # Save the results to a new shapefile
    watersheds.to_file(output_path)
    print("Shapefile exported")

In [ ]:
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import fiona

In [ ]:
def rasterizeVector(vector_file, output_raster_path):

    # Define the resolution (30m x 30m)
    resolution = 0.000278 # in meters

    # Define the column name to be used for rasterization
    attribute_column = "DD"

    # Read the vector shapefile
    with fiona.open(vector_file, "r") as shapefile:
        shapes = [(feature["geometry"], feature["properties"][attribute_column]) for feature in shapefile]
        bounds = shapefile.bounds
        print(f"Shapefile bounds: {bounds}")
        if bounds is None:
            # Calculate bounds manually
            all_geometries = [feature["geometry"] for feature in shapefile]
            bounds = fiona.collection.bounds(all_geometries)
            print(f"Calculated bounds: {bounds}")
    # Define the raster metadata
    minx, miny, maxx, maxy = bounds
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)
    transform = from_origin(minx, maxy, resolution, resolution)
    # Rasterize the vector shapes based on the attribute column
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        fill=0,  # Background value
        transform=transform,
        all_touched=True,
        dtype=rasterio.float32  # or another appropriate data type
    )

    # Save the raster to a file
    with rasterio.open(
        output_raster_path, 'w',
        driver='GTiff',
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=shapefile.crs,
        transform=transform,
    ) as dst:
        dst.write(raster, 1)

    print(f"Rasterized shapefile saved as {output_raster_path}")



In [ ]:
def main():
    # fixed paths
    # need to update acc to drive
    basin_boundary_path = '/content/drive/MyDrive/ICTD_Datasets/Hydrological boundaries/Basins_Ankit/CGWB_basin.shp' # path to cgqb basins india shape file
    drainage_lines_folder = '/content/drive/MyDrive/ICTD_Datasets/Hydrological boundaries/Drainage_Lines_Basin_Ankit' # path to folder containing basin wise drainage lines

    # Assumption : The input vector shapefile of any block must be of exterior
    #               mws boundary shapefile, else we would need to first get the exterior mws
    #               boundary shapefile for that block and continue

    # path to the input vector shapefile of the block to compute DD
    input_block_shapefile_path = '/content/drive/MyDrive/Ankit_2023MCS2485/mohanpur.shp'

    clipped_dl_path = '/content/drive/MyDrive/Ankit_2023MCS2485/mohanpur_dl.shp'
    clip_drainage_lines(basin_boundary_path, input_block_shapefile_path, drainage_lines_folder, clipped_dl_path)

    dd_vector_path = '/content/drive/MyDrive/Ankit_2023MCS2485/mohanpur_dd.shp'
    calculate_drainage_density(input_block_shapefile_path, clipped_dl_path, dd_vector_path)

    dd_raster_path = '/content/drive/MyDrive/Ankit_2023MCS2485/mohanpur_dd.tif'
    rasterizeVector(dd_vector_path, dd_raster_path)

In [ ]:
if __name__ == "__main__":
    main()